<a href="https://colab.research.google.com/github/rajendranaidu495/OneClassClassification/blob/main/CMS_bk_03212023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. All Datasets: https://data.cms.gov/provider-summary-by-type-of-service/
2. CMS Part D datasets: : https://data.cms.gov/provider-summary-by-type-of-service/medicare-part-d-prescribers 
3. CMS Part B datasets: https://data.cms.gov/provider-summary-by-type-of-service/medicare-physician-other-practitioners

4. DMEPOS (Medicare Durable Medical Equipment, Devices & Supplies - by Referring Provider and Service): https://data.cms.gov/provider-summary-by-type-of-service/medicare-durable-medical-equipment-devices-supplies/medicare-durable-medical-equipment-devices-supplies-by-referring-provider-and-service

5. LEIE Databases https://oig.hhs.gov/exclusions/exclusions_list.asp

Research paper : https://journalofbigdata.springeropen.com/articles/10.1186/s40537-018-0138-3#auth-Matthew-Herland

To connect with goole drive for datasets. <br>
Gmail : rajendramannam9@gmail.com <br>
password: Mlds@2020

old_colab link: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
!pip install pandasql
import pandasql as ps
#from pandasql import sqldf
#import pandasql as ps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26787 sha256=a8f270dc198180c4bf602eaeb98de8f16f5294208c1d794cfa536b50a2bef1e0
  Stored in directory: /root/.cache/pip/wheels/ed/8f/46/a383923333728744f01ba24adbd8e364f2cb9470a8b8e5b9ff
Successfully built pandasql


In [ ]:
partD_dataset_path = "/content/drive/MyDrive/CMS/partD/MUP_DPR_RY21_P04_V10_DY19_NPIBN_1.csv"
partb_dataset_path = "/content/drive/MyDrive/CMS/partb/MUP_PHY_R21_P04_V10_D19_Prov_Svc.csv"
DMEPOS_dataset_path = "/content/drive/MyDrive/CMS/DMEPOS/mup_dme_ry21_p05_v10_dy19_prvhpr_0.csv"
LEIE_dataset_path = "/content/drive/MyDrive/CMS/LEIE/UPDATED.csv"

In [ ]:
# #read data in chunks of 1 million rows at a time 
# #partD
# chunk_partd = pd.read_csv(partD_dataset_path, encoding = "ISO-8859-1" ,chunksize=1000000)
# PartD_dataset_org= pd.concat(chunk_partd)

# #partb 
# chunk_partb = pd.read_csv(partb_dataset_path, encoding = "ISO-8859-1" ,chunksize=1000000)
# PartB_dataset_org= pd.concat(chunk_partb)

# # DMEPOS
# chunk_DMEPOS_dataset = pd.read_csv(DMEPOS_dataset_path, encoding = "ISO-8859-1" ,chunksize=1000000)
# DMEPOS_dataset_org= pd.concat(chunk_DMEPOS_dataset)

# # LEIE
# chunk_LEIE_dataset = pd.read_csv(LEIE_dataset_path, encoding = "ISO-8859-1" ,chunksize=1000000)
# LEIE_dataset_org= pd.concat(chunk_LEIE_dataset)

Data processing 

In [ ]:
# Calendar year 2019 
# select only required columns
# Remove all instances that either were missing both NPI and HCPCS/drug name values in all datasets.
#pd.set_option("display.max_columns", None)
#LEIE_dataset_org.head()

In [ ]:
#partb 
chunk_partb = pd.read_csv(partb_dataset_path, nrows=109999, encoding = "ISO-8859-1" ,chunksize=10000)
#chunk_partb = pd.read_csv(partb_dataset_path, encoding = "ISO-8859-1" ,chunksize=10000)
PartB_dataset_org= pd.concat(chunk_partb)

In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)
PartB_dataset_org.head()

,Rndrng_NPI,Rndrng_Prvdr_Last_Org_Name,Rndrng_Prvdr_First_Name,Rndrng_Prvdr_MI,Rndrng_Prvdr_Crdntls,Rndrng_Prvdr_Gndr,Rndrng_Prvdr_Ent_Cd,Rndrng_Prvdr_St1,Rndrng_Prvdr_St2,Rndrng_Prvdr_City,Rndrng_Prvdr_State_Abrvtn,Rndrng_Prvdr_State_FIPS,Rndrng_Prvdr_Zip5,Rndrng_Prvdr_RUCA,Rndrng_Prvdr_RUCA_Desc,Rndrng_Prvdr_Cntry,Rndrng_Prvdr_Type,Rndrng_Prvdr_Mdcr_Prtcptg_Ind,HCPCS_Cd,HCPCS_Desc,HCPCS_Drug_Ind,Place_Of_Srvc,Tot_Benes,Tot_Srvcs,Tot_Bene_Day_Srvcs,Avg_Sbmtd_Chrg,Avg_Mdcr_Alowd_Amt,Avg_Mdcr_Pymt_Amt,Avg_Mdcr_Stdzd_Amt
0,1003000126,Enkeshafi,Ardalan,NaN,M.D.,M,I,900 Seton Dr,NaN,Cumberland,MD,24.0,21502,1.0,Metropolitan area core: primary flow within an...,US,Internal Medicine,Y,99217,Hospital observation care on day of discharge,N,F,40,40.0,40,232.275000,72.59000,57.870000,58.19175
1,1003000126,Enkeshafi,Ardalan,NaN,M.D.,M,I,900 Seton Dr,NaN,Cumberland,MD,24.0,21502,1.0,Metropolitan area core: primary flow within an...,US,Internal Medicine,Y,99220,"Hospital observation care, typically 70 minutes",N,F,25,25.0,25,712.800000,186.72520,148.851600,147.77000
2,1003000126,Enkeshafi,Ardalan,NaN,M.D.,M,I,900 Seton Dr,NaN,Cumberland,MD,24.0,21502,1.0,Metropolitan area core: primary flow within an...,US,Internal Medicine,Y,99221,"Initial hospital inpatient care, typically 30 ...",N,F,24,24.0,24,320.166667,101.12125,80.615833,80.81000
3,1003000126,Enkeshafi,Ardalan,NaN,M.D.,M,I,900 Seton Dr,NaN,Cumberland,MD,24.0,21502,1.0,Metropolitan area core: primary flow within an...,US,Internal Medicine,Y,99223,"Initial hospital inpatient care, typically 70 ...",N,F,157,158.0,158,651.417722,200.93000,160.190000,161.06000
4,1003000126,Enkeshafi,Ardalan,NaN,M.D.,M,I,900 Seton Dr,NaN,Cumberland,MD,24.0,21502,1.0,Metropolitan area core: primary flow within an...,US,Internal Medicine,Y,99232,"Subsequent hospital inpatient care, typically ...",N,F,117,205.0,205,245.614634,72.75639,58.005561,57.93000


In [ ]:
# dataset =  NPI, codes 
# dataset = naidu , [codeA, codeb, code c , etc ]
# embedding with universal encoder 
# dataset = naidu , [4*300]
# dataset = naidu , 4*300 , fraud(0/1)

# x = 4*300 y = 1 
# dataset split 
#model = cnn(x,y)
##cnn (x_tr, y_yr)
# 

In [ ]:
# For Part B, the aggregating process consists of grouping the data by NPI, provider type, gender and year, aggregating over HCPCS and place of service.

#Npi,Provider_type ,Nppes_provider_gender ,Line_srvc_cnt ,Bene_unique_cnt ,Bene_day_srvc_cnt ,Average_submitted_chrg_amt ,Average_medicare_payment_amt 
PartB_Required_columns = [ 'Rndrng_NPI', 'Rndrng_Prvdr_Crdntls',  'Rndrng_Prvdr_Gndr' ,'Tot_Srvcs', 'Tot_Benes' , 'Tot_Bene_Day_Srvcs', 'Avg_Sbmtd_Chrg', 'Avg_Mdcr_Pymt_Amt', 'Place_Of_Srvc', 'HCPCS_Cd' ]
PartB_df = PartB_dataset_org[PartB_Required_columns].iloc[:100000]

In [ ]:
#q1 = "select * from PartB_df"
#abc = ps.sqldf(q1)
PartB_df.head()

,Rndrng_NPI,Rndrng_Prvdr_Crdntls,Rndrng_Prvdr_Gndr,Tot_Srvcs,Tot_Benes,Tot_Bene_Day_Srvcs,Avg_Sbmtd_Chrg,Avg_Mdcr_Pymt_Amt,Place_Of_Srvc,HCPCS_Cd
0,1003000126,M.D.,M,40.0,40,40,232.275000,57.870000,F,99217
1,1003000126,M.D.,M,25.0,25,25,712.800000,148.851600,F,99220
2,1003000126,M.D.,M,24.0,24,24,320.166667,80.615833,F,99221
3,1003000126,M.D.,M,158.0,157,158,651.417722,160.190000,F,99223
4,1003000126,M.D.,M,205.0,117,205,245.614634,58.005561,F,99232


In [ ]:
partb_groupby_df = PartB_df.groupby(['Rndrng_NPI',\
                                     'Rndrng_Prvdr_Crdntls',\
                                     'Avg_Sbmtd_Chrg',\
                                     'Avg_Mdcr_Pymt_Amt',\
                                     'Avg_Sbmtd_Chrg',\
                                     'Place_Of_Srvc',\
                                     'HCPCS_Cd']\
                                     )\
                                      .agg({
                                          'Tot_Benes' : ['sum', 'mean', 'median', np.std, 'min', 'max'],\
                                          'Tot_Bene_Day_Srvcs' : ['sum', 'mean', 'median', np.std, 'min', 'max'],\
                                          'Avg_Mdcr_Pymt_Amt' : ['sum', 'mean', 'median', np.std, 'min', 'max'],\
                                          'Avg_Sbmtd_Chrg' : ['sum', 'mean', 'median', np.std, 'min', 'max']
                                      })

In [ ]:
partb_groupby_df.head()

Tot_Benes  \
                                                                                                             sum   
Rndrng_NPI Rndrng_Prvdr_Crdntls Avg_Sbmtd_Chrg Avg_Mdcr_Pymt_Amt Avg_Sbmtd_Chrg Place_Of_Srvc HCPCS_Cd             
1003000126 M.D.                 229.475000     57.870000         229.475000     F             99238           39   
                                232.275000     57.870000         232.275000     F             99217           40   
                                245.614634     58.005561         245.614634     F             99232          117   
                                320.166667     80.615833         320.166667     F             99221           24   
                                345.236364     82.588165         345.236364     F             99233          306   

                                                                                                               \
                                                                                                         mean   
Rndrng_NPI Rndrng_Prvdr_Crdntls Avg_Sbmtd_Chrg Avg_Mdcr_Pymt_Amt Avg_Sbmtd_Chrg Place_Of_Srvc HCPCS_Cd          
1003000126 M.D.                 229.475000     57.870000         229.475000     F             99238      39.0   
                                232.275000     57.870000         232.275000     F             99217      40.0   
                                245.614634     58.005561         245.614634     F             99232     117.0   
                                320.166667     80.615833         320.166667     F             99221      24.0   
                                345.236364     82.588165         345.236364     F             99233     306.0   

                                                                                                               \
                                                                                                       median   
Rndrng_NPI Rndrng_Prvdr_Crdntls Avg_Sbmtd_Chrg Avg_Mdcr_Pymt_Amt Avg_Sbmtd_Chrg Place_Of_Srvc HCPCS_Cd          
1003000126 M.D.                 229.475000     57.870000         229.475000     F             99238      39.0   
                                232.275000     57.870000         232.275000     F             99217      40.0   
                                245.614634     58.005561         245.614634     F             99232     117.0   
                                320.166667     80.615833         320.166667     F             99221      24.0   
                                345.236364     82.588165         345.236364     F             99233     306.0   

                                                                                                            \
                                                                                                       std   
Rndrng_NPI Rndrng_Prvdr_Crdntls Avg_Sbmtd_Chrg Avg_Mdcr_Pymt_Amt Avg_Sbmtd_Chrg Place_Of_Srvc HCPCS_Cd       
1003000126 M.D.                 229.475000     57.870000         229.475000     F             99238    NaN   
                                232.275000     57.870000         232.275000     F             99217    NaN   
                                245.614634     58.005561         245.614634     F             99232    NaN   
                                320.166667     80.615833         320.166667     F             99221    NaN   
                                345.236364     82.588165         345.236364     F             99233    NaN   

                                                                                                             \
                                                                                                        min   
Rndrng_NPI Rndrng_Prvdr_Crdntls Avg_Sbmtd_Chrg Avg_Mdcr_Pymt_Amt Avg_Sbmtd_Chrg Place_Of_Srvc HCPCS_Cd        
1003000126 M.D.                 229.475000     57.870000         229.475000     F             99238      39   
                                232.275

In [ ]:
partb_groupby_df.columns = [''.join(col) for col in partb_groupby_df.columns.values]

In [ ]:
partb_groupby_df.head()

Tot_Benessum  \
Rndrng_NPI Rndrng_Prvdr_Crdntls Avg_Sbmtd_Chrg Avg_Mdcr_Pymt_Amt Avg_Sbmtd_Chrg Place_Of_Srvc HCPCS_Cd                 
1003000126 M.D.                 229.475000     57.870000         229.475000     F             99238               39   
                                232.275000     57.870000         232.275000     F             99217               40   
                                245.614634     58.005561         245.614634     F             99232              117   
                                320.166667     80.615833         320.166667     F             99221               24   
                                345.236364     82.588165         345.236364     F             99233              306   

                                                                                                        Tot_Benesmean  \
Rndrng_NPI Rndrng_Prvdr_Crdntls Avg_Sbmtd_Chrg Avg_Mdcr_Pymt_Amt Avg_Sbmtd_Chrg Place_Of_Srvc HCPCS_Cd                  
1003000126 M.D.                 229.475000     57.870000         229.475000     F             99238              39.0   
                                232.275000     57.870000         232.275000     F             99217              40.0   
                                245.614634     58.005561         245.614634     F             99232             117.0   
                                320.166667     80.615833         320.166667     F             99221              24.0   
                                345.236364     82.588165         345.236364     F             99233             306.0   

                                                                                                        Tot_Benesmedian  \
Rndrng_NPI Rndrng_Prvdr_Crdntls Avg_Sbmtd_Chrg Avg_Mdcr_Pymt_Amt Avg_Sbmtd_Chrg Place_Of_Srvc HCPCS_Cd                    
1003000126 M.D.                 229.475000     57.870000         229.475000     F             99238                39.0   
                                232.275000     57.870000         232.275000     F             99217                40.0   
                                245.614634     58.005561         245.614634     F             99232               117.0   
                                320.166667     80.615833         320.166667     F             99221                24.0   
                                345.236364     82.588165         345.236364     F             99233               306.0   

                                                                                                        Tot_Benesstd  \
Rndrng_NPI Rndrng_Prvdr_Crdntls Avg_Sbmtd_Chrg Avg_Mdcr_Pymt_Amt Avg_Sbmtd_Chrg Place_Of_Srvc HCPCS_Cd                 
1003000126 M.D.                 229.475000     57.870000         229.475000     F             99238              NaN   
                                232.275000     57.870000         232.275000     F             99217              NaN   
                                245.614634     58.005561         245.614634     F             99232              NaN   
                                320.166667     80.615833         320.166667     F             99221              NaN   
                                345.236364     82.588165         345.236364     F             99233              NaN   

                                                                                                        Tot_Benesmin  \
Rndrng_NPI Rndrng_Prvdr_Crdntls Avg_Sbmtd_Chrg Avg_Mdcr_Pymt_Amt Avg_Sbmtd_Chrg Place_Of_Srvc HCPCS_Cd                 
1003000126 M.D.                 229.475000     57.870000         229.475000     F             99238               39   
                                232.275000     57.870000         232.275000     F             99217               40   
                                245.614634     58.005561         245.614634     F             99232              117   
                                320.166667     80.615833         320.166667     F             99221             

In [ ]:
#partb_groupby_df.reset_index()
partb_groupby_df = partb_groupby_df.reset_index(level = ['Rndrng_NPI', 'Rndrng_Prvdr_Crdntls','Place_Of_Srvc'])

In [ ]:
partb_groupby_df.head()

,,,,Rndrng_NPI,Rndrng_Prvdr_Crdntls,Place_Of_Srvc,Tot_Benessum,Tot_Benesmean,Tot_Benesmedian,Tot_Benesstd,Tot_Benesmin,Tot_Benesmax,Tot_Bene_Day_Srvcssum,Tot_Bene_Day_Srvcsmean,Tot_Bene_Day_Srvcsmedian,Tot_Bene_Day_Srvcsstd,Tot_Bene_Day_Srvcsmin,Tot_Bene_Day_Srvcsmax,Avg_Mdcr_Pymt_Amtsum,Avg_Mdcr_Pymt_Amtmean,Avg_Mdcr_Pymt_Amtmedian,Avg_Mdcr_Pymt_Amtstd,Avg_Mdcr_Pymt_Amtmin,Avg_Mdcr_Pymt_Amtmax,Avg_Sbmtd_Chrgsum,Avg_Sbmtd_Chrgmean,Avg_Sbmtd_Chrgmedian,Avg_Sbmtd_Chrgstd,Avg_Sbmtd_Chrgmin,Avg_Sbmtd_Chrgmax
Avg_Sbmtd_Chrg,Avg_Mdcr_Pymt_Amt,Avg_Sbmtd_Chrg,HCPCS_Cd,,,,,,,,,,,,,,,,,,,,,,,,,,,
229.475000,57.870000,229.475000,99238,1003000126,M.D.,F,39,39.0,39.0,NaN,39,39,40,40.0,40.0,NaN,40,40,57.870000,57.870000,57.870000,NaN,57.870000,57.870000,229.475000,229.475000,229.475000,NaN,229.475000,229.475000
232.275000,57.870000,232.275000,99217,1003000126,M.D.,F,40,40.0,40.0,NaN,40,40,40,40.0,40.0,NaN,40,40,57.870000,57.870000,57.870000,NaN,57.870000,57.870000,232.275000,232.275000,232.275000,NaN,232.275000,232.275000
245.614634,58.005561,245.614634,99232,1003000126,M.D.,F,117,117.0,117.0,NaN,117,117,205,205.0,205.0,NaN,205,205,58.005561,58.005561,58.005561,NaN,58.005561,58.005561,245.614634,245.614634,245.614634,NaN,245.614634,245.614634
320.166667,80.615833,320.166667,99221,1003000126,M.D.,F,24,24.0,24.0,NaN,24,24,24,24.0,24.0,NaN,24,24,80.615833,80.615833,80.615833,NaN,80.615833,80.615833,320.166667,320.166667,320.166667,NaN,320.166667,320.166667
345.236364,82.588165,345.236364,99233,1003000126,M.D.,F,306,306.0,306.0,NaN,306,306,605,605.0,605.0,NaN,605,605,82.588165,82.588165,82.588165,NaN,82.588165,82.588165,345.236364,345.236364,345.236364,NaN,345.236364,345.236364


#Part D

In [ ]:
#partD
chunk_partd = pd.read_csv(partD_dataset_path, nrows=259999, encoding = "ISO-8859-1" ,chunksize=1000)
PartD_dataset_org= pd.concat(chunk_partd)

In [ ]:
pd.set_option("display.max_columns", None)
PartD_dataset_org.head()

,Prscrbr_NPI,Prscrbr_Last_Org_Name,Prscrbr_First_Name,Prscrbr_City,Prscrbr_State_Abrvtn,Prscrbr_State_FIPS,Prscrbr_Type,Prscrbr_Type_Src,Brnd_Name,Gnrc_Name,Tot_Clms,Tot_30day_Fills,Tot_Day_Suply,Tot_Drug_Cst,Tot_Benes,GE65_Sprsn_Flag,GE65_Tot_Clms,GE65_Tot_30day_Fills,GE65_Tot_Drug_Cst,GE65_Tot_Day_Suply,GE65_Bene_Sprsn_Flag,GE65_Tot_Benes
0,1003000126,Enkeshafi,Ardalan,Cumberland,MD,24,Internal Medicine,S,Amiodarone Hcl,Amiodarone Hcl,13,13.0,360,150.76,NaN,NaN,13.0,13.0,150.76,360.0,*,NaN
1,1003000126,Enkeshafi,Ardalan,Cumberland,MD,24,Internal Medicine,S,Amlodipine Besylate,Amlodipine Besylate,22,26.0,780,92.81,12.0,NaN,22.0,26.0,92.81,780.0,NaN,12.0
2,1003000126,Enkeshafi,Ardalan,Cumberland,MD,24,Internal Medicine,S,Atorvastatin Calcium,Atorvastatin Calcium,41,50.0,1482,373.60,22.0,NaN,41.0,50.0,373.60,1482.0,NaN,22.0
3,1003000126,Enkeshafi,Ardalan,Cumberland,MD,24,Internal Medicine,S,Azithromycin,Azithromycin,15,15.0,77,132.57,15.0,#,NaN,NaN,NaN,NaN,#,NaN
4,1003000126,Enkeshafi,Ardalan,Cumberland,MD,24,Internal Medicine,S,Carvedilol,Carvedilol,21,27.0,787,118.53,11.0,NaN,21.0,27.0,118.53,787.0,NaN,11.0


In [ ]:
# Part D was grouped by NPI, provider type and year aggregating over drugs. 

# keep Npi, Specialty_description, Bene_count, Total_claim_count, Total_30_day_fill_count, Total_day_supply, Total_drug_cost, Exclusion 
part_d_required_columns = [ 'Prscrbr_NPI', 'Prscrbr_Type' , 'Tot_Benes' , 'Tot_Clms' , 'Tot_30day_Fills', 'Tot_Day_Suply' , 'Tot_Drug_Cst' ]
PartD_df =PartD_dataset_org[part_d_required_columns].loc[:1000000]

In [ ]:
partd_bk =PartD_df

In [ ]:
df_partd = PartD_df.groupby(["Prscrbr_NPI",\
                             "Prscrbr_Type"])\
                                  .agg({
                                      'Tot_Clms' : ['sum', 'mean', 'median', np.std, 'min', 'max'],\
                                      'Tot_30day_Fills' : ['sum', 'mean', 'median', np.std, 'min', 'max'],\
                                      'Tot_Day_Suply' : ['sum', 'mean', 'median', np.std, 'min', 'max'],\
                                      'Tot_Drug_Cst' : ['sum', 'mean', 'median', np.std, 'min', 'max']
                                  })
 

In [ ]:
df_partd.head()

Tot_Clms                                   \
                                         sum       mean median        std min   
Prscrbr_NPI Prscrbr_Type                                                        
1003000126  Internal Medicine            336  18.666667   15.0   8.317239  11   
1003000142  Anesthesiology               972  60.750000   34.5  61.505013  13   
1003000167  Dentist                       43  21.500000   21.5  10.606602  14   
1003000282  Nurse Practitioner            12  12.000000   12.0        NaN  12   
1003000423  Obstetrics & Gynecology      106  17.666667   17.0   6.022181  11   

                                         Tot_30day_Fills                    \
                                     max             sum       mean median   
Prscrbr_NPI Prscrbr_Type                                                     
1003000126  Internal Medicine         41           405.0  22.500000   17.5   
1003000142  Anesthesiology           249          1027.1  64.193750   37.0   
1003000167  Dentist                   29            43.0  21.500000   21.5   
1003000282  Nurse Practitioner        12            14.0  14.000000   14.0   
1003000423  Obstetrics & Gynecology   25           170.5  28.416667   26.0   

                                                            Tot_Day_Suply  \
                                           std   min    max           sum   
Prscrbr_NPI Prscrbr_Type                                                    
1003000126  Internal Medicine        10.700467  12.0   50.0          9957   
1003000142  Anesthesiology           68.805547  14.0  284.5         29078   
1003000167  Dentist                  10.606602  14.0   29.0           222   
1003000282  Nurse Practitioner             NaN  14.0   14.0           420   
1003000423  Obstetrics & Gynecology  15.988798  13.0   53.5          4515   

                                                                            \
                                            mean  median          std  min   
Prscrbr_NPI Prscrbr_Type                                                     
1003000126  Internal Medicine         553.166667   471.0   393.154849   73   
1003000142  Anesthesiology           1817.375000  1098.5  2013.956401  420   
1003000167  Dentist                   111.000000   111.0    19.798990   97   
1003000282  Nurse Practitioner        420.000000   420.0          NaN  420   
1003000423  Obstetrics & Gynecology   752.500000   647.0   564.163717  164   

                                          Tot_Drug_Cst                        \
                                      max          sum         mean   median   
Prscrbr_NPI Prscrbr_Type                                                       
1003000126  Internal Medicine        1482      9798.06   544.336667  150.980   
1003000142  Anesthesiology           8407     44952.80  2809.550000  833.055   
1003000167  Dentist                   125       169.49    84.745000   84.745   
1003000282  Nurse Practitioner        420        95.03    95.030000   95.030   
1003000423  Obstetrics & Gynecology  1582     14671.45  2445.241667  619.985   

                                                                    
                                             std     min       max  
Prscrbr_NPI Prscrbr_Type                                            
1003000126  Internal Medicine        1671.251796   40.25   7234.67  
1003000142  Anesthesiology           6232.854718   99.18  25818.67  
1003000167  Dentist                     6.512453   80.14     89.35  
1003000282  Nurse Practitioner               NaN   95.03     95.03  
1003000423  Obstetrics & Gynecology  3704.954292  172.77   9592.06

In [ ]:
df_partd.columns = [''.join(col) for col in df_partd.columns.values]

In [ ]:
df_partd.head()

,,Tot_Clmssum,Tot_Clmsmean,Tot_Clmsmedian,Tot_Clmsstd,Tot_Clmsmin,Tot_Clmsmax,Tot_30day_Fillssum,Tot_30day_Fillsmean,Tot_30day_Fillsmedian,Tot_30day_Fillsstd,Tot_30day_Fillsmin,Tot_30day_Fillsmax,Tot_Day_Suplysum,Tot_Day_Suplymean,Tot_Day_Suplymedian,Tot_Day_Suplystd,Tot_Day_Suplymin,Tot_Day_Suplymax,Tot_Drug_Cstsum,Tot_Drug_Cstmean,Tot_Drug_Cstmedian,Tot_Drug_Cststd,Tot_Drug_Cstmin,Tot_Drug_Cstmax
Prscrbr_NPI,Prscrbr_Type,,,,,,,,,,,,,,,,,,,,,,,,
1003000126,Internal Medicine,336,18.666667,15.0,8.317239,11,41,405.0,22.500000,17.5,10.700467,12.0,50.0,9957,553.166667,471.0,393.154849,73,1482,9798.06,544.336667,150.980,1671.251796,40.25,7234.67
1003000142,Anesthesiology,972,60.750000,34.5,61.505013,13,249,1027.1,64.193750,37.0,68.805547,14.0,284.5,29078,1817.375000,1098.5,2013.956401,420,8407,44952.80,2809.550000,833.055,6232.854718,99.18,25818.67
1003000167,Dentist,43,21.500000,21.5,10.606602,14,29,43.0,21.500000,21.5,10.606602,14.0,29.0,222,111.000000,111.0,19.798990,97,125,169.49,84.745000,84.745,6.512453,80.14,89.35
1003000282,Nurse Practitioner,12,12.000000,12.0,NaN,12,12,14.0,14.000000,14.0,NaN,14.0,14.0,420,420.000000,420.0,NaN,420,420,95.03,95.030000,95.030,NaN,95.03,95.03
1003000423,Obstetrics & Gynecology,106,17.666667,17.0,6.022181,11,25,170.5,28.416667,26.0,15.988798,13.0,53.5,4515,752.500000,647.0,564.163717,164,1582,14671.45,2445.241667,619.985,3704.954292,172.77,9592.06


In [ ]:
#partb_groupby_df.reset_index()
df_partd = df_partd.reset_index(level = ['Prscrbr_NPI'])

# DMEPOS

In [ ]:
# DMEPOS
chunk_DMEPOS_dataset = pd.read_csv(DMEPOS_dataset_path, nrows=999999, encoding = "ISO-8859-1" ,chunksize=1000000)
DMEPOS_dataset_org= pd.concat(chunk_DMEPOS_dataset)

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: DtypeWarning: Columns (11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)


In [ ]:
#DMEPOS was grouped by NPI, provider type, gender and year, aggregating over HCPCS and rental status. 

# DMEPOS_dataset
# Referring_npi ,Referring_provider_type  ,Referring_provider_gender ,Number_of_suppliers ,Number_of_supplier_beneficiaries ,Number_of_supplier_claims ,
#Number_of_supplier_services ,Avg_supplier_submitted_charge ,Avg_supplier_medicare_pmt_amt 

DMEPOS_Required_columns = [ 'Rfrg_NPI', 'Rfrg_Prvdr_Crdntls' , 'Rfrg_Prvdr_Gndr', 'Tot_Suplrs', 'Tot_Suplr_Benes', 'Tot_Suplr_Clms', 'Tot_Suplr_Srvcs', 'Avg_Suplr_Sbmtd_Chrg', 'Avg_Suplr_Mdcr_Pymt_Amt' ]
DMEPOS_df = DMEPOS_dataset_org [DMEPOS_Required_columns].iloc[:100000]

In [ ]:
pd.set_option("display.max_columns", None)
DMEPOS_df.head()

,Rfrg_NPI,Rfrg_Prvdr_Crdntls,Rfrg_Prvdr_Gndr,Tot_Suplrs,Tot_Suplr_Benes,Tot_Suplr_Clms,Tot_Suplr_Srvcs,Avg_Suplr_Sbmtd_Chrg,Avg_Suplr_Mdcr_Pymt_Amt
0,1003000126,M.D.,M,5,NaN,69,69,52.207536,17.732029
1,1003000126,M.D.,M,5,12.0,88,88,362.305227,94.287500
2,1003000522,MD,M,15,22.0,55,132,61.314242,5.757803
3,1003000522,MD,M,2,NaN,13,13,14.158462,2.686923
4,1003000522,MD,M,8,13.0,22,30,13.608667,0.979333


In [ ]:
df_DMEPOS = DMEPOS_df.groupby(["Rfrg_NPI",\
                               'Rfrg_Prvdr_Crdntls',\
                               'Tot_Suplrs', \
                               'Tot_Suplr_Benes',\
                               'Tot_Suplr_Clms',\
                               'Tot_Suplr_Srvcs',\
                               'Avg_Suplr_Sbmtd_Chrg',\
                               'Avg_Suplr_Mdcr_Pymt_Amt',\
                             ])\
                                  .agg({
                                      'Tot_Suplrs' : ['sum', 'mean', 'median', np.std, 'min', 'max'],\
                                      'Tot_Suplr_Benes' : ['sum', 'mean', 'median', np.std, 'min', 'max'],\
                                      'Tot_Suplr_Clms' : ['sum', 'mean', 'median', np.std, 'min', 'max'],\
                                      'Tot_Suplr_Srvcs' : ['sum', 'mean', 'median', np.std, 'min', 'max'],\
                                      'Avg_Suplr_Sbmtd_Chrg' : ['sum', 'mean', 'median', np.std, 'min', 'max'],\
                                      'Avg_Suplr_Mdcr_Pymt_Amt' : ['sum', 'mean', 'median', np.std, 'min', 'max'],\

                                  })
 

In [ ]:
df_DMEPOS.head()

Tot_Suplrs  \
                                                                                                                                            sum   
Rfrg_NPI   Rfrg_Prvdr_Crdntls Tot_Suplrs Tot_Suplr_Benes Tot_Suplr_Clms Tot_Suplr_Srvcs Avg_Suplr_Sbmtd_Chrg Avg_Suplr_Mdcr_Pymt_Amt              
1003000126 M.D.               5          12.0            88             88              362.305227           94.287500                        5   
1003000522 MD                 8          13.0            22             30              13.608667            0.979333                         8   
                              15         22.0            55             132             61.314242            5.757803                        15   
1003000902 MD                 12         11.0            21             23              14.003043            1.025652                        12   
                              17         22.0            57             124             71.633468            5.433790                        17   

                                                                                                                                            \
                                                                                                                                      mean   
Rfrg_NPI   Rfrg_Prvdr_Crdntls Tot_Suplrs Tot_Suplr_Benes Tot_Suplr_Clms Tot_Suplr_Srvcs Avg_Suplr_Sbmtd_Chrg Avg_Suplr_Mdcr_Pymt_Amt         
1003000126 M.D.               5          12.0            88             88              362.305227           94.287500                 5.0   
1003000522 MD                 8          13.0            22             30              13.608667            0.979333                  8.0   
                              15         22.0            55             132             61.314242            5.757803                 15.0   
1003000902 MD                 12         11.0            21             23              14.003043            1.025652                 12.0   
                              17         22.0            57             124             71.633468            5.433790                 17.0   

                                                                                                                                             \
                                                                                                                                     median   
Rfrg_NPI   Rfrg_Prvdr_Crdntls Tot_Suplrs Tot_Suplr_Benes Tot_Suplr_Clms Tot_Suplr_Srvcs Avg_Suplr_Sbmtd_Chrg Avg_Suplr_Mdcr_Pymt_Amt          
1003000126 M.D.               5          12.0            88             88              362.305227           94.287500                  5.0   
1003000522 MD                 8          13.0            22             30              13.608667            0.979333                   8.0   
                              15         22.0            55             132             61.314242            5.757803                  15.0   
1003000902 MD                 12         11.0            21             23              14.003043            1.025652                  12.0   
                              17         22.0            57             124             71.633468            5.433790                  17.0   

                                                                                                                                          \
                                                                                                                                     std   
Rfrg_NPI   Rfrg_Prvdr_Crdntls Tot_Suplrs Tot_Suplr_Benes Tot_Suplr_Clms Tot_Suplr_Srvcs Avg_Suplr_Sbmtd_Chrg Avg_Suplr_Mdcr_Pymt_Amt       
1003000126 M.D.               5          12.0            88             88              362.305227           94.287500               NaN   
1003000522 MD                 8          13.0            22             30              13.608667            0.979

# LEIE

In [ ]:
# LEIE
chunk_LEIE_dataset = pd.read_csv(LEIE_dataset_path, encoding = "ISO-8859-1" ,chunksize=1000000)
LEIE_dataset_org= pd.concat(chunk_LEIE_dataset)

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)


In [ ]:
pd.set_option("display.max_columns", None)
LEIE_dataset_org.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE
0,NaN,NaN,,"#1 MARKETING SERVICE, INC",OTHER BUSINESS,SOBER HOME,NaN,0,NaN,239 BRIGHTON BEACH AVENUE,BROOKLYN,NY,11235,1128a1,20200319,0,0,NaN
1,NaN,NaN,,101 FIRST CARE PHARMACY INC,OTHER BUSINESS,PHARMACY,NaN,1972902351,NaN,"C/O 609 W 191ST STREET, APT D",NEW YORK,NY,10040,1128b8,20220320,0,0,NaN
2,NaN,NaN,,14 LAWRENCE AVE PHARMACY,PHARMACY,NaN,NaN,0,NaN,14 LAWRENCE AVENUE,SMITHTOWN,NY,11787,1128a1,19880830,0,0,NaN
3,NaN,NaN,,143 MEDICAL EQUIPMENT CO,DME COMPANY,DME - OXYGEN,NaN,0,NaN,701 NW 36 AVENUE,MIAMI,FL,33125,1128b7,19970620,0,0,NaN
4,NaN,NaN,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,20180419,0,0,NaN


In [ ]:
LEIE_Required_columns = [ 'NPI', 'EXCLTYPE' , 'EXCLDATE' , 'REINDATE']
LEIE_df = LEIE_dataset_org[LEIE_Required_columns]

In [ ]:
LEIE_df_a= LEIE_df[LEIE_df['NPI'] != 0]

In [ ]:
len(LEIE_df_a)

6384

In [ ]:
LEIE_df_a.head()

,NPI,EXCLTYPE,EXCLDATE,REINDATE
1,1972902351,1128b8,20220320,0
4,1922348218,1128a1,20180419,0
24,1942476080,1128b8,20170518,0
28,1275600959,1128a1,20130320,0
31,1891731758,1128b8,20170518,0


In [ ]:
exclusion_codes = ['1128a1', '1128a2', '1128a3', '1128b4', '1128b7', '1128c3gi', '1128c3gii']
filtered_leie = LEIE_df_a[LEIE_df_a.EXCLTYPE.isin(exclusion_codes)]

In [ ]:
len(filtered_leie)

4668

In [ ]:
filtered_leie.head()

,NPI,EXCLTYPE,EXCLDATE,REINDATE
4,1922348218,1128a1,20180419,0
28,1275600959,1128a1,20130320,0
41,1265830335,1128a1,20220818,0
61,1851631543,1128b7,20190326,0
62,1902198435,1128a1,20160120,0


In [ ]:
filtered_leie['EXCLDATE_YEAR'] = pd.to_datetime(filtered_leie.EXCLDATE, format='%Y%m%d') 

<ipython-input-40-2640bbd7450c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_leie['EXCLDATE_YEAR'] = pd.to_datetime(filtered_leie.EXCLDATE, format='%Y%m%d')


In [ ]:
filtered_leie.head()

,NPI,EXCLTYPE,EXCLDATE,REINDATE,EXCLDATE_YEAR
4,1922348218,1128a1,20180419,0,2018-04-19
28,1275600959,1128a1,20130320,0,2013-03-20
41,1265830335,1128a1,20220818,0,2022-08-18
61,1851631543,1128b7,20190326,0,2019-03-26
62,1902198435,1128a1,20160120,0,2016-01-20


In [ ]:
filtered_leie['END_EXCLDATE'] = filtered_leie['EXCLDATE_YEAR'].dt.year

<ipython-input-42-4f5ae2ef2f2a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_leie['END_EXCLDATE'] = filtered_leie['EXCLDATE_YEAR'].dt.year


In [ ]:
filtered_leie.head()

,NPI,EXCLTYPE,EXCLDATE,REINDATE,EXCLDATE_YEAR,END_EXCLDATE
4,1922348218,1128a1,20180419,0,2018-04-19,2018
28,1275600959,1128a1,20130320,0,2013-03-20,2013
41,1265830335,1128a1,20220818,0,2022-08-18,2022
61,1851631543,1128b7,20190326,0,2019-03-26,2019
62,1902198435,1128a1,20160120,0,2016-01-20,2016


In [ ]:
filtered_leie_a = filtered_leie.loc[filtered_leie['END_EXCLDATE'] == 2018]

In [ ]:
filtered_leie_a.head()

,NPI,EXCLTYPE,EXCLDATE,REINDATE,EXCLDATE_YEAR,END_EXCLDATE
4,1922348218,1128a1,20180419,0,2018-04-19,2018
248,1275856452,1128a1,20180118,0,2018-01-18,2018
345,1437449139,1128a1,20180719,0,2018-07-19,2018
471,1710102033,1128a1,20180320,0,2018-03-20,2018
521,1184859761,1128b7,20181016,0,2018-10-16,2018


In [ ]:
len(filtered_leie_a)

344

In [ ]:
filtered_leie_a['target'] = 1

<ipython-input-47-95f08d5cd3db>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_leie_a['target'] = 1


In [ ]:
filtered_leie_a.head()

,NPI,EXCLTYPE,EXCLDATE,REINDATE,EXCLDATE_YEAR,END_EXCLDATE,target
4,1922348218,1128a1,20180419,0,2018-04-19,2018,1
248,1275856452,1128a1,20180118,0,2018-01-18,2018,1
345,1437449139,1128a1,20180719,0,2018-07-19,2018,1
471,1710102033,1128a1,20180320,0,2018-03-20,2018,1
521,1184859761,1128b7,20181016,0,2018-10-16,2018,1


In [ ]:
Target_df = filtered_leie_a[['NPI','target']]

In [ ]:
Target_df.head()

,NPI,target
4,1922348218,1
248,1275856452,1
345,1437449139,1
471,1710102033,1
521,1184859761,1


In [ ]:
Target_df.dtypes

NPI       int64
target    int64
dtype: object

In [ ]:
len(Target_df)

344

In [ ]:
len(LEIE_df_a)

6384

In [ ]:
Final_withoutcodes_leie = LEIE_df_a[['NPI']]

In [ ]:
Final_withoutcodes_leie.head()

,NPI
1,1972902351
4,1922348218
24,1942476080
28,1275600959
31,1891731758


In [ ]:
Final_withoutcodes_leie['target'] = 1

<ipython-input-56-d91152302741>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_withoutcodes_leie['target'] = 1


In [ ]:
Final_withoutcodes_leie.head()

,NPI,target
1,1972902351,1
4,1922348218,1
24,1942476080,1
28,1275600959,1
31,1891731758,1


In [ ]:
#LEIE_df_a['dateformatchange'] = pd.to_datetime(LEIE_df_a.EXCLDATE, format ='%Y%m%d' )

In [ ]:
#LEIE_df_a['START_EXCLDATE'] = LEIE_df_a['dateformatchange'].dt.year

In [ ]:
# combined all databases 
# The combining process involves a join operation on NPI, provider type, and year. 

In [ ]:
combined = pd.merge(partb_groupby_df,df_partd, 
                    left_on=['Rndrng_NPI'], 
                    right_on=['Prscrbr_NPI']).merge(df_DMEPOS, 
                                                  left_on=['Rndrng_NPI'], 
                                                  right_on=['Rfrg_NPI'])

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:9190: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  return merge(


In [ ]:
combined.head()

,Rndrng_NPI,Rndrng_Prvdr_Crdntls,Place_Of_Srvc,Tot_Benessum,Tot_Benesmean,Tot_Benesmedian,Tot_Benesstd,Tot_Benesmin,Tot_Benesmax,Tot_Bene_Day_Srvcssum,Tot_Bene_Day_Srvcsmean,Tot_Bene_Day_Srvcsmedian,Tot_Bene_Day_Srvcsstd,Tot_Bene_Day_Srvcsmin,Tot_Bene_Day_Srvcsmax,Avg_Mdcr_Pymt_Amtsum,Avg_Mdcr_Pymt_Amtmean,Avg_Mdcr_Pymt_Amtmedian,Avg_Mdcr_Pymt_Amtstd,Avg_Mdcr_Pymt_Amtmin,Avg_Mdcr_Pymt_Amtmax,Avg_Sbmtd_Chrgsum,Avg_Sbmtd_Chrgmean,Avg_Sbmtd_Chrgmedian,Avg_Sbmtd_Chrgstd,Avg_Sbmtd_Chrgmin,Avg_Sbmtd_Chrgmax,Prscrbr_NPI,Tot_Clmssum,Tot_Clmsmean,Tot_Clmsmedian,Tot_Clmsstd,Tot_Clmsmin,Tot_Clmsmax,Tot_30day_Fillssum,Tot_30day_Fillsmean,Tot_30day_Fillsmedian,Tot_30day_Fillsstd,Tot_30day_Fillsmin,Tot_30day_Fillsmax,Tot_Day_Suplysum,Tot_Day_Suplymean,Tot_Day_Suplymedian,Tot_Day_Suplystd,Tot_Day_Suplymin,Tot_Day_Suplymax,Tot_Drug_Cstsum,Tot_Drug_Cstmean,Tot_Drug_Cstmedian,Tot_Drug_Cststd,Tot_Drug_Cstmin,Tot_Drug_Cstmax,"(Tot_Suplrs, sum)","(Tot_Suplrs, mean)","(Tot_Suplrs, median)","(Tot_Suplrs, std)","(Tot_Suplrs, min)","(Tot_Suplrs, max)","(Tot_Suplr_Benes, sum)","(Tot_Suplr_Benes, mean)","(Tot_Suplr_Benes, median)","(Tot_Suplr_Benes, std)","(Tot_Suplr_Benes, min)","(Tot_Suplr_Benes, max)","(Tot_Suplr_Clms, sum)","(Tot_Suplr_Clms, mean)","(Tot_Suplr_Clms, median)","(Tot_Suplr_Clms, std)","(Tot_Suplr_Clms, min)","(Tot_Suplr_Clms, max)","(Tot_Suplr_Srvcs, sum)","(Tot_Suplr_Srvcs, mean)","(Tot_Suplr_Srvcs, median)","(Tot_Suplr_Srvcs, std)","(Tot_Suplr_Srvcs, min)","(Tot_Suplr_Srvcs, max)","(Avg_Suplr_Sbmtd_Chrg, sum)","(Avg_Suplr_Sbmtd_Chrg, mean)","(Avg_Suplr_Sbmtd_Chrg, median)","(Avg_Suplr_Sbmtd_Chrg, std)","(Avg_Suplr_Sbmtd_Chrg, min)","(Avg_Suplr_Sbmtd_Chrg, max)","(Avg_Suplr_Mdcr_Pymt_Amt, sum)","(Avg_Suplr_Mdcr_Pymt_Amt, mean)","(Avg_Suplr_Mdcr_Pymt_Amt, median)","(Avg_Suplr_Mdcr_Pymt_Amt, std)","(Avg_Suplr_Mdcr_Pymt_Amt, min)","(Avg_Suplr_Mdcr_Pymt_Amt, max)"
0,1003000126,M.D.,F,39,39.0,39.0,NaN,39,39,40,40.0,40.0,NaN,40,40,57.870000,57.870000,57.870000,NaN,57.870000,57.870000,229.475000,229.475000,229.475000,NaN,229.475000,229.475000,1003000126,336,18.666667,15.0,8.317239,11,41,405.0,22.5,17.5,10.700467,12.0,50.0,9957,553.166667,471.0,393.154849,73,1482,9798.06,544.336667,150.98,1671.251796,40.25,7234.67,5,5.0,5.0,NaN,5,5,12.0,12.0,12.0,NaN,12.0,12.0,88,88.0,88.0,NaN,88,88,88,88.0,88.0,NaN,88,88,362.305227,362.305227,362.305227,NaN,362.305227,362.305227,94.2875,94.2875,94.2875,NaN,94.2875,94.2875
1,1003000126,M.D.,F,40,40.0,40.0,NaN,40,40,40,40.0,40.0,NaN,40,40,57.870000,57.870000,57.870000,NaN,57.870000,57.870000,232.275000,232.275000,232.275000,NaN,232.275000,232.275000,1003000126,336,18.666667,15.0,8.317239,11,41,405.0,22.5,17.5,10.700467,12.0,50.0,9957,553.166667,471.0,393.154849,73,1482,9798.06,544.336667,150.98,1671.251796,40.25,7234.67,5,5.0,5.0,NaN,5,5,12.0,12.0,12.0,NaN,12.0,12.0,88,88.0,88.0,NaN,88,88,88,88.0,88.0,NaN,88,88,362.305227,362.305227,362.305227,NaN,362.305227,362.305227,94.2875,94.2875,94.2875,NaN,94.2875,94.2875
2,1003000126,M.D.,F,117,117.0,117.0,NaN,117,117,205,205.0,205.0,NaN,205,205,58.005561,58.005561,58.005561,NaN,58.005561,58.005561,245.614634,245.614634,245.614634,NaN,245.614634,245.614634,1003000126,336,18.666667,15.0,8.317239,11,41,405.0,22.5,17.5,10.700467,12.0,50.0,9957,553.166667,471.0,393.154849,73,1482,9798.06,544.336667,150.98,1671.251796,40.25,7234.67,5,5.0,5.0,NaN,5,5,12.0,12.0,12.0,NaN,12.0,12.0,88,88.0,88.0,NaN,88,88,88,88.0,88.0,NaN,88,88,362.305227,362.305227,362.305227,NaN,362.305227,362.305227,94.2875,94.2875,94.2875,NaN,94.2875,94.2875
3,1003000126,M.D.,F,24,24.0,24.0,NaN,24,24,24,24.0,24.0,NaN,24,24,80.615833,80.615833,80.615833,NaN,80.615833,80.615833,320.166667,320.166667,320.166667,NaN,320.166667,320.166667,1003000126,336,18.666667,15.0,8.317239,11,41,405.0,22.5,17.5,10.700467,12.0,50.0,9957,553.166667,471.0,393.154849,73,1482,9798.06,544.336667,150.98,1671.251796,40.25,7234.67,5,5.0,5.0,NaN,5,5,12.0,12.0,12.0,NaN,12.0,12.0,88,88.0,88.0,NaN,88,88,88,88.0,88.0,NaN,88,88,362.305227,3

In [ ]:
from pandas.core.reshape.merge import string
# merge with target lables

combined = combined.fillna(0)
Final_withoutcodes_leie = Final_withoutcodes_leie.fillna(0)
final_df = pd.merge(combined, Final_withoutcodes_leie, how = 'outer'
                    , left_on = 'Rndrng_NPI',
                    right_on = ['NPI'])
final_df = final_df.fillna(0)
final_df.Rndrng_NPI = final_df.Rndrng_NPI.astype(int)
final_df.NPI = final_df.NPI.astype(int)
final_df.target = final_df.target.astype(int)


In [ ]:
# change the categorical into numerical for columns : Rndrng_Prvdr_Crdntls , Place_Of_Srvc
final_df['Rndrng_Prvdr_Crdntls'] = pd.Categorical(final_df['Rndrng_Prvdr_Crdntls']).codes

In [ ]:
final_df['Place_Of_Srvc'] = pd.Categorical(final_df['Place_Of_Srvc']).codes

In [ ]:
final_df.head()

,Rndrng_NPI,Rndrng_Prvdr_Crdntls,Place_Of_Srvc,Tot_Benessum,Tot_Benesmean,Tot_Benesmedian,Tot_Benesstd,Tot_Benesmin,Tot_Benesmax,Tot_Bene_Day_Srvcssum,Tot_Bene_Day_Srvcsmean,Tot_Bene_Day_Srvcsmedian,Tot_Bene_Day_Srvcsstd,Tot_Bene_Day_Srvcsmin,Tot_Bene_Day_Srvcsmax,Avg_Mdcr_Pymt_Amtsum,Avg_Mdcr_Pymt_Amtmean,Avg_Mdcr_Pymt_Amtmedian,Avg_Mdcr_Pymt_Amtstd,Avg_Mdcr_Pymt_Amtmin,Avg_Mdcr_Pymt_Amtmax,Avg_Sbmtd_Chrgsum,Avg_Sbmtd_Chrgmean,Avg_Sbmtd_Chrgmedian,Avg_Sbmtd_Chrgstd,Avg_Sbmtd_Chrgmin,Avg_Sbmtd_Chrgmax,Prscrbr_NPI,Tot_Clmssum,Tot_Clmsmean,Tot_Clmsmedian,Tot_Clmsstd,Tot_Clmsmin,Tot_Clmsmax,Tot_30day_Fillssum,Tot_30day_Fillsmean,Tot_30day_Fillsmedian,Tot_30day_Fillsstd,Tot_30day_Fillsmin,Tot_30day_Fillsmax,Tot_Day_Suplysum,Tot_Day_Suplymean,Tot_Day_Suplymedian,Tot_Day_Suplystd,Tot_Day_Suplymin,Tot_Day_Suplymax,Tot_Drug_Cstsum,Tot_Drug_Cstmean,Tot_Drug_Cstmedian,Tot_Drug_Cststd,Tot_Drug_Cstmin,Tot_Drug_Cstmax,"(Tot_Suplrs, sum)","(Tot_Suplrs, mean)","(Tot_Suplrs, median)","(Tot_Suplrs, std)","(Tot_Suplrs, min)","(Tot_Suplrs, max)","(Tot_Suplr_Benes, sum)","(Tot_Suplr_Benes, mean)","(Tot_Suplr_Benes, median)","(Tot_Suplr_Benes, std)","(Tot_Suplr_Benes, min)","(Tot_Suplr_Benes, max)","(Tot_Suplr_Clms, sum)","(Tot_Suplr_Clms, mean)","(Tot_Suplr_Clms, median)","(Tot_Suplr_Clms, std)","(Tot_Suplr_Clms, min)","(Tot_Suplr_Clms, max)","(Tot_Suplr_Srvcs, sum)","(Tot_Suplr_Srvcs, mean)","(Tot_Suplr_Srvcs, median)","(Tot_Suplr_Srvcs, std)","(Tot_Suplr_Srvcs, min)","(Tot_Suplr_Srvcs, max)","(Avg_Suplr_Sbmtd_Chrg, sum)","(Avg_Suplr_Sbmtd_Chrg, mean)","(Avg_Suplr_Sbmtd_Chrg, median)","(Avg_Suplr_Sbmtd_Chrg, std)","(Avg_Suplr_Sbmtd_Chrg, min)","(Avg_Suplr_Sbmtd_Chrg, max)","(Avg_Suplr_Mdcr_Pymt_Amt, sum)","(Avg_Suplr_Mdcr_Pymt_Amt, mean)","(Avg_Suplr_Mdcr_Pymt_Amt, median)","(Avg_Suplr_Mdcr_Pymt_Amt, std)","(Avg_Suplr_Mdcr_Pymt_Amt, min)","(Avg_Suplr_Mdcr_Pymt_Amt, max)",NPI,target
0,1003000126,42,1,39.0,39.0,39.0,0.0,39.0,39.0,40.0,40.0,40.0,0.0,40.0,40.0,57.870000,57.870000,57.870000,0.0,57.870000,57.870000,229.475000,229.475000,229.475000,0.0,229.475000,229.475000,1.003000e+09,336.0,18.666667,15.0,8.317239,11.0,41.0,405.0,22.5,17.5,10.700467,12.0,50.0,9957.0,553.166667,471.0,393.154849,73.0,1482.0,9798.06,544.336667,150.98,1671.251796,40.25,7234.67,5.0,5.0,5.0,0.0,5.0,5.0,12.0,12.0,12.0,0.0,12.0,12.0,88.0,88.0,88.0,0.0,88.0,88.0,88.0,88.0,88.0,0.0,88.0,88.0,362.305227,362.305227,362.305227,0.0,362.305227,362.305227,94.2875,94.2875,94.2875,0.0,94.2875,94.2875,0,0
1,1003000126,42,1,40.0,40.0,40.0,0.0,40.0,40.0,40.0,40.0,40.0,0.0,40.0,40.0,57.870000,57.870000,57.870000,0.0,57.870000,57.870000,232.275000,232.275000,232.275000,0.0,232.275000,232.275000,1.003000e+09,336.0,18.666667,15.0,8.317239,11.0,41.0,405.0,22.5,17.5,10.700467,12.0,50.0,9957.0,553.166667,471.0,393.154849,73.0,1482.0,9798.06,544.336667,150.98,1671.251796,40.25,7234.67,5.0,5.0,5.0,0.0,5.0,5.0,12.0,12.0,12.0,0.0,12.0,12.0,88.0,88.0,88.0,0.0,88.0,88.0,88.0,88.0,88.0,0.0,88.0,88.0,362.305227,362.305227,362.305227,0.0,362.305227,362.305227,94.2875,94.2875,94.2875,0.0,94.2875,94.2875,0,0
2,1003000126,42,1,117.0,117.0,117.0,0.0,117.0,117.0,205.0,205.0,205.0,0.0,205.0,205.0,58.005561,58.005561,58.005561,0.0,58.005561,58.005561,245.614634,245.614634,245.614634,0.0,245.614634,245.614634,1.003000e+09,336.0,18.666667,15.0,8.317239,11.0,41.0,405.0,22.5,17.5,10.700467,12.0,50.0,9957.0,553.166667,471.0,393.154849,73.0,1482.0,9798.06,544.336667,150.98,1671.251796,40.25,7234.67,5.0,5.0,5.0,0.0,5.0,5.0,12.0,12.0,12.0,0.0,12.0,12.0,88.0,88.0,88.0,0.0,88.0,88.0,88.0,88.0,88.0,0.0,88.0,88.0,362.305227,362.305227,362.305227,0.0,362.305227,362.305227,94.2875,94.2875,94.2875,0.0,94.2875,94.2875,0,0
3,1003000126,42,1,24.0,24.0,24.0,0.0,24.0,24.0,24.0,24.0,24.0,0.0,24.0,24.0,80.615833,80.615833,80.615833,0.0,80.615833,80.615833,320.166667,320.166667,320.166667,0.0,320.166667,320.166667,1.003000e+09,336.0,18.666667,15.0,8.317239,11.0,41.0,405.0,22.5,17.5,10.700467,12.0,50.0,9957.0,553.166667,471.0,393.154

In [ ]:
# df['C'] = pd.concat([df['A'], df['B']], axis=0)

#final_df['NPI_final'] = pd.concat([final_df['Rndrng_NPI'], final_df['NPI']], axis =0)

ValueError: ignored

In [ ]:
#one hot encoding for categorical data

#combined_category_columns = ['Rndrng_Prvdr_Crdntls', 'Place_Of_Srvc']
#final_df = pd.get_dummies(final_df, columns=combined_category_columns, drop_first=True)

In [ ]:
final_df.dtypes

Rndrng_NPI                          int64
Rndrng_Prvdr_Crdntls                 int8
Place_Of_Srvc                        int8
Tot_Benessum                      float64
Tot_Benesmean                     float64
                                   ...   
(Avg_Suplr_Mdcr_Pymt_Amt, std)    float64
(Avg_Suplr_Mdcr_Pymt_Amt, min)    float64
(Avg_Suplr_Mdcr_Pymt_Amt, max)    float64
NPI                                 int64
target                              int64
Length: 90, dtype: object

In [ ]:
final_df.head()

,Rndrng_NPI,Rndrng_Prvdr_Crdntls,Place_Of_Srvc,Tot_Benessum,Tot_Benesmean,Tot_Benesmedian,Tot_Benesstd,Tot_Benesmin,Tot_Benesmax,Tot_Bene_Day_Srvcssum,Tot_Bene_Day_Srvcsmean,Tot_Bene_Day_Srvcsmedian,Tot_Bene_Day_Srvcsstd,Tot_Bene_Day_Srvcsmin,Tot_Bene_Day_Srvcsmax,Avg_Mdcr_Pymt_Amtsum,Avg_Mdcr_Pymt_Amtmean,Avg_Mdcr_Pymt_Amtmedian,Avg_Mdcr_Pymt_Amtstd,Avg_Mdcr_Pymt_Amtmin,Avg_Mdcr_Pymt_Amtmax,Avg_Sbmtd_Chrgsum,Avg_Sbmtd_Chrgmean,Avg_Sbmtd_Chrgmedian,Avg_Sbmtd_Chrgstd,Avg_Sbmtd_Chrgmin,Avg_Sbmtd_Chrgmax,Prscrbr_NPI,Tot_Clmssum,Tot_Clmsmean,Tot_Clmsmedian,Tot_Clmsstd,Tot_Clmsmin,Tot_Clmsmax,Tot_30day_Fillssum,Tot_30day_Fillsmean,Tot_30day_Fillsmedian,Tot_30day_Fillsstd,Tot_30day_Fillsmin,Tot_30day_Fillsmax,Tot_Day_Suplysum,Tot_Day_Suplymean,Tot_Day_Suplymedian,Tot_Day_Suplystd,Tot_Day_Suplymin,Tot_Day_Suplymax,Tot_Drug_Cstsum,Tot_Drug_Cstmean,Tot_Drug_Cstmedian,Tot_Drug_Cststd,Tot_Drug_Cstmin,Tot_Drug_Cstmax,"(Tot_Suplrs, sum)","(Tot_Suplrs, mean)","(Tot_Suplrs, median)","(Tot_Suplrs, std)","(Tot_Suplrs, min)","(Tot_Suplrs, max)","(Tot_Suplr_Benes, sum)","(Tot_Suplr_Benes, mean)","(Tot_Suplr_Benes, median)","(Tot_Suplr_Benes, std)","(Tot_Suplr_Benes, min)","(Tot_Suplr_Benes, max)","(Tot_Suplr_Clms, sum)","(Tot_Suplr_Clms, mean)","(Tot_Suplr_Clms, median)","(Tot_Suplr_Clms, std)","(Tot_Suplr_Clms, min)","(Tot_Suplr_Clms, max)","(Tot_Suplr_Srvcs, sum)","(Tot_Suplr_Srvcs, mean)","(Tot_Suplr_Srvcs, median)","(Tot_Suplr_Srvcs, std)","(Tot_Suplr_Srvcs, min)","(Tot_Suplr_Srvcs, max)","(Avg_Suplr_Sbmtd_Chrg, sum)","(Avg_Suplr_Sbmtd_Chrg, mean)","(Avg_Suplr_Sbmtd_Chrg, median)","(Avg_Suplr_Sbmtd_Chrg, std)","(Avg_Suplr_Sbmtd_Chrg, min)","(Avg_Suplr_Sbmtd_Chrg, max)","(Avg_Suplr_Mdcr_Pymt_Amt, sum)","(Avg_Suplr_Mdcr_Pymt_Amt, mean)","(Avg_Suplr_Mdcr_Pymt_Amt, median)","(Avg_Suplr_Mdcr_Pymt_Amt, std)","(Avg_Suplr_Mdcr_Pymt_Amt, min)","(Avg_Suplr_Mdcr_Pymt_Amt, max)",NPI,target
0,1003000126,42,1,39.0,39.0,39.0,0.0,39.0,39.0,40.0,40.0,40.0,0.0,40.0,40.0,57.870000,57.870000,57.870000,0.0,57.870000,57.870000,229.475000,229.475000,229.475000,0.0,229.475000,229.475000,1.003000e+09,336.0,18.666667,15.0,8.317239,11.0,41.0,405.0,22.5,17.5,10.700467,12.0,50.0,9957.0,553.166667,471.0,393.154849,73.0,1482.0,9798.06,544.336667,150.98,1671.251796,40.25,7234.67,5.0,5.0,5.0,0.0,5.0,5.0,12.0,12.0,12.0,0.0,12.0,12.0,88.0,88.0,88.0,0.0,88.0,88.0,88.0,88.0,88.0,0.0,88.0,88.0,362.305227,362.305227,362.305227,0.0,362.305227,362.305227,94.2875,94.2875,94.2875,0.0,94.2875,94.2875,0,0
1,1003000126,42,1,40.0,40.0,40.0,0.0,40.0,40.0,40.0,40.0,40.0,0.0,40.0,40.0,57.870000,57.870000,57.870000,0.0,57.870000,57.870000,232.275000,232.275000,232.275000,0.0,232.275000,232.275000,1.003000e+09,336.0,18.666667,15.0,8.317239,11.0,41.0,405.0,22.5,17.5,10.700467,12.0,50.0,9957.0,553.166667,471.0,393.154849,73.0,1482.0,9798.06,544.336667,150.98,1671.251796,40.25,7234.67,5.0,5.0,5.0,0.0,5.0,5.0,12.0,12.0,12.0,0.0,12.0,12.0,88.0,88.0,88.0,0.0,88.0,88.0,88.0,88.0,88.0,0.0,88.0,88.0,362.305227,362.305227,362.305227,0.0,362.305227,362.305227,94.2875,94.2875,94.2875,0.0,94.2875,94.2875,0,0
2,1003000126,42,1,117.0,117.0,117.0,0.0,117.0,117.0,205.0,205.0,205.0,0.0,205.0,205.0,58.005561,58.005561,58.005561,0.0,58.005561,58.005561,245.614634,245.614634,245.614634,0.0,245.614634,245.614634,1.003000e+09,336.0,18.666667,15.0,8.317239,11.0,41.0,405.0,22.5,17.5,10.700467,12.0,50.0,9957.0,553.166667,471.0,393.154849,73.0,1482.0,9798.06,544.336667,150.98,1671.251796,40.25,7234.67,5.0,5.0,5.0,0.0,5.0,5.0,12.0,12.0,12.0,0.0,12.0,12.0,88.0,88.0,88.0,0.0,88.0,88.0,88.0,88.0,88.0,0.0,88.0,88.0,362.305227,362.305227,362.305227,0.0,362.305227,362.305227,94.2875,94.2875,94.2875,0.0,94.2875,94.2875,0,0
3,1003000126,42,1,24.0,24.0,24.0,0.0,24.0,24.0,24.0,24.0,24.0,0.0,24.0,24.0,80.615833,80.615833,80.615833,0.0,80.615833,80.615833,320.166667,320.166667,320.166667,0.0,320.166667,320.166667,1.003000e+09,336.0,18.666667,15.0,8.317239,11.0,41.0,405.0,22.5,17.5,10.700467,12.0,50.0,9957.0,553.166667,471.0,393.154

Train/Test split

In [ ]:
#X_train_combined, X_test_combined, y_train_combined, y_test_combined = train_test_split(x_combined, y_combined, test_size=0.2, random_state=0)

In [ ]:
final_df.head()

In [ ]:
final_df1 = final_df.drop_duplicates()

In [ ]:
final_df1.head()

In [ ]:
pd.to_pickle(final_df, '/content/final_df.pkl')

In [ ]:
pip install seaborn

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.countplot(x = "target", data = final_df)

In [ ]:
#combined dataset
X_combined = final_df.drop(columns=['target'], axis=1)
y_combined = final_df.target

In [ ]:
len(X_combined)

In [ ]:
from imblearn.over_sampling import SMOTE
import numpy as np

# Initialize the SMOTE object
sm = SMOTE(random_state=42)


In [ ]:
# Apply the SMOTE technique to the dataset
X_combined, y_combined = sm.fit_resample(X_combined, y_combined)

In [ ]:
sns.countplot(x = y_combined)

##Train/Test split

In [ ]:
X_train_combined, X_test_combined, y_train_combined, y_test_combined = train_test_split(X_combined, y_combined, test_size=0.2, random_state=0)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler


%matplotlib inline
pd.options.display.max_rows = 1000

In [ ]:
# scale
combined_feature_names = X_combined.columns.to_list()

X_combined_scaler = StandardScaler()
X_combined_scaled = X_combined_scaler.fit_transform(X_train_combined)

X_train_combined = pd.DataFrame(X_combined_scaled, columns=combined_feature_names)
X_test_combined = pd.DataFrame(X_combined_scaler.transform(X_test_combined), columns = combined_feature_names)

#hyper-parameters

In [ ]:
#Logistic Regression
#logreg_combined = LogisticRegression(max_iter=10**4, penalty='l1', solver='saga')



In [ ]:
#param_grid_logreg = {  
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': [.01, .05, .001],
}

In [ ]:
#grid_tree_logreg_combined = RandomizedSearchCV(logreg_combined, 
                                  param_grid_logreg, 
                                  cv=5,
                                  n_jobs=-1,
                                  scoring='f1')

In [ ]:
#grid_tree_logreg_combined.fit(X_train_combined, y_train_combined)

In [ ]:
#grid_tree_logreg_combined.best_params_

In [ ]:
#y_pred_combined = grid_tree_logreg_combined.predict(X_test_combined)

In [ ]:
#confusion_matrix_combined = metrics.plot_confusion_matrix(grid_tree_logreg_combined, X_test_combined, y_test_combined, cmap= "GnBu", ax = ax[0,0], normalize='all')

fig, ax = plt.subplots(2,2)
fig.set_size_inches(15, 11)

ax[0,0].set_title('Combined Dataset')
ax[0,0].set_xticklabels(['NOT FRAUD','FRAUD'])
ax[0,0].set_yticklabels(['NOT FRAUD','FRAUD'])

In [ ]:
# Feature Importances

In [ ]:
# apply OC-SVM
# Train the one class support vector machine (SVM) model
# Model and performance
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
one_class_svm = OneClassSVM(nu=0.01, kernel = 'rbf', gamma = 'auto').fit(X_train_combined)

In [ ]:
# Predict the anomalies
prediction = one_class_svm.predict(X_test_combined)
# Change the anomalies' values to make it consistent with the true values
prediction = [1 if i==-1 else 0 for i in prediction]
# Check the model performance
print(classification_report(y_test_combined, prediction))